In [23]:
import numpy as np 
import pandas as pd 
import os
import optuna
import pickle
import datetime

#### Skin Sensetization 

In [2]:
knime_pos = pd.read_csv("../data/skin-sensitization/knime-qsar/pos.smiles.knime.csv")
knime_summ = pd.read_csv("../data/skin-sensitization/knime-qsar/pos.smiles_Summary_file.csv")

In [3]:
knime_pos.head()

,CASRN,SMILES
0,1655500-83-6,CC(CC=C(C)C)C1CC1(C)CO
1,109-88-6,[Mg++].C[O-].C[O-]
2,77-78-1,COS(=O)(=O)OC
3,2082-81-7,CC(=C)C(=O)OCCCCOC(=O)C(C)=C
4,75-33-2,CC(C)S


In [4]:
knime_summ.head()

,RowID,Molecule name,Original_SMILES,Number of connected components,Canonical_QSARr,InChI_Code_QSARr,InChI Key_QSARr,Salt_Solvent,Salt_Solvent_ID
0,1,Molecule_1,CC(CC=C(C)C)C1CC1(C)CO,1,CC(CC=C(C)C)C1CC1(C)CO,InChI=1S/C12H22O/c1-9(2)5-6-10(3)11-7-12(114)8...,FZLOGXXTGWFQFP-UHFFFAOYSA-N,NaN,0
1,2,Molecule_2,[Mg+2].C[O-].C[O-],3,CO,InChI=1S/CH4O/c1-2/h2H1H3,OKKJLVBELUTLKV-UHFFFAOYSA-N,[Mg+2],120
2,3,Molecule_3,COS(=O)(=O)OC,1,COS(=O)(=O)OC,InChI=1S/C2H6O4S/c1-5-7(34)6-2/h1-2H3,VAYGXNSJCAHWJZ-UHFFFAOYSA-N,NaN,0
3,4,Molecule_4,CC(=C)C(=O)OCCCCOC(=O)C(C)=C,1,CC(=C)C(=O)OCCCCOC(=O)C(C)=C,InChI=1S/C12H18O4/c1-9(2)11(13)15-7-5-6-8-16-1...,XOJWAAUYNWGQAU-UHFFFAOYSA-N,NaN,0
4,5,Molecule_5,CC(C)S,1,CC(C)S,InChI=1S/C3H8S/c1-3(2)4/h3-4H1-2H3,KJRCEJOSASVSRA-UHFFFAOYSA-N,NaN,0


In [5]:
knime_pos['RowID'] = knime_pos.index + 1 

In [6]:
knime_pos.head()

,CASRN,SMILES,RowID
0,1655500-83-6,CC(CC=C(C)C)C1CC1(C)CO,1
1,109-88-6,[Mg++].C[O-].C[O-],2
2,77-78-1,COS(=O)(=O)OC,3
3,2082-81-7,CC(=C)C(=O)OCCCCOC(=O)C(C)=C,4
4,75-33-2,CC(C)S,5


In [7]:
knime_pos.shape, knime_summ.shape

((2741, 3), (2435, 9))

####  Merging KNIME SMILES and the Summary SMILES data

In [8]:
merge_knime = knime_summ.merge(knime_pos, on = 'RowID', how= 'left')

In [9]:
merge_knime.head()

,RowID,Molecule name,Original_SMILES,Number of connected components,Canonical_QSARr,InChI_Code_QSARr,InChI Key_QSARr,Salt_Solvent,Salt_Solvent_ID,CASRN,SMILES
0,1,Molecule_1,CC(CC=C(C)C)C1CC1(C)CO,1,CC(CC=C(C)C)C1CC1(C)CO,InChI=1S/C12H22O/c1-9(2)5-6-10(3)11-7-12(114)8...,FZLOGXXTGWFQFP-UHFFFAOYSA-N,NaN,0,1655500-83-6,CC(CC=C(C)C)C1CC1(C)CO
1,2,Molecule_2,[Mg+2].C[O-].C[O-],3,CO,InChI=1S/CH4O/c1-2/h2H1H3,OKKJLVBELUTLKV-UHFFFAOYSA-N,[Mg+2],120,109-88-6,[Mg++].C[O-].C[O-]
2,3,Molecule_3,COS(=O)(=O)OC,1,COS(=O)(=O)OC,InChI=1S/C2H6O4S/c1-5-7(34)6-2/h1-2H3,VAYGXNSJCAHWJZ-UHFFFAOYSA-N,NaN,0,77-78-1,COS(=O)(=O)OC
3,4,Molecule_4,CC(=C)C(=O)OCCCCOC(=O)C(C)=C,1,CC(=C)C(=O)OCCCCOC(=O)C(C)=C,InChI=1S/C12H18O4/c1-9(2)11(13)15-7-5-6-8-16-1...,XOJWAAUYNWGQAU-UHFFFAOYSA-N,NaN,0,2082-81-7,CC(=C)C(=O)OCCCCOC(=O)C(C)=C
4,5,Molecule_5,CC(C)S,1,CC(C)S,InChI=1S/C3H8S/c1-3(2)4/h3-4H1-2H3,KJRCEJOSASVSRA-UHFFFAOYSA-N,NaN,0,75-33-2,CC(C)S


In [10]:
merge_knime.shape

(2435, 11)

In [11]:
print("Smiles matches = ",np.sum(merge_knime['Original_SMILES'] == merge_knime['SMILES']))

Smiles matches =  713


In [12]:
merge_knime[np.logical_not(merge_knime['Original_SMILES'] == merge_knime['SMILES'])]

,RowID,Molecule name,Original_SMILES,Number of connected components,Canonical_QSARr,InChI_Code_QSARr,InChI Key_QSARr,Salt_Solvent,Salt_Solvent_ID,CASRN,SMILES
1,2,Molecule_2,[Mg+2].C[O-].C[O-],3,CO,InChI=1S/CH4O/c1-2/h2H1H3,OKKJLVBELUTLKV-UHFFFAOYSA-N,[Mg+2],120,109-88-6,[Mg++].C[O-].C[O-]
6,7,Molecule_7,[Na+].[O-]S(=O)(=O)C1C=CC(=CC=1)NC1N=C(Cl)N=C(...,2,OS(=O)(=O)C1C=CC(=CC=1)NC1N=C(Cl)N=C(Cl)N=1,InChI=1S/C9H6Cl2N4O3S/c10-7-13-8(11)15-9(14-7)...,JDBGDMZILOWAHV-UHFFFAOYSA-N,[Na+],110,4156-21-2,[Na+].[O-]S(=O)(=O)C1=CC=C(NC2=NC(Cl)=NC(Cl)=N...
7,9,Molecule_9,[Mg+2].CCOC([O-])=O.CCOC([O-])=O,3,CCOC(O)=O,InChI=1S/C3H6O3/c1-2-6-3(4)5/h2H21H3(H45),CQDGTJPVBWZJAZ-UHFFFAOYSA-N,[Mg+2],120,16891-37-5,[Mg++].CCOC([O-])=O.CCOC([O-])=O
8,10,Molecule_10,CC(C)=CCCC(C)=CC=O,1,CC(C)=CCCC(C)=CC=O,InChI=1S/C10H16O/c1-9(2)5-4-6-10(3)7-8-11/h57-...,WTEVQBCEXWBHNA-UHFFFAOYSA-N,NaN,0,106-26-3,CC(C)=CCC\C(C)=C/C=O
9,11,Molecule_11,Cl.NC1=CC=C(CCNCC(O)C2C=CC=CC=2)C=C1,2,NC1=CC=C(CCNCC(O)C2C=CC=CC=2)C=C1,InChI=1S/C16H20N2O/c17-15-8-6-13(7-9-15)10-11-...,TUAHDMSPHZSMQN-UHFFFAOYSA-N,Cl,170,521284-22-0,Cl.NC1=CC=C(CCNC[C@H](O)C2=CC=CC=C2)C=C1
...,...,...,...,...,...,...,...,...,...,...,...
2429,2731,Molecule_2731,[Zn+2].CCCCN(CCCC)C([S-])=S.CCCCN(CCCC)C([S-])=S,3,CCCCN(CCCC)C(S)=S,InChI=1S/C9H19NS2/c1-3-5-7-10(9(11)12)8-6-4-2/...,SZRLKIKBPASKQH-UHFFFAOYSA-N,[Zn+2],300,136-23-2,[Zn++].CCCCN(CCCC)C([S-])=S.CCCCN(CCCC)C([S-])=S
2430,2732,Molecule_2732,[Zn+2].CCN(CC)C([S-])=S.CCN(CC)C([S-])=S,3,CCN(CC)C(S)=S,InChI=1S/C5H11NS2/c1-3-6(4-2)5(7)8/h3-4H21-2H3...,LMBWSYZSUOEYSN-UHFFFAOYSA-N,[Zn+2],300,14324-55-1,[Zn++].CCN(CC)C([S-])=S.CCN(CC)C([S-])=S
2431,2734,Molecule_2734,[Zn+2].[S-]C(=S)NCCNC([S-])=S,2,SC(=S)NCCNC(S)=S,InChI=1S/C4H8N2S4/c7-3(8)5-1-2-6-4(9)10/h1-2H2...,AWYFNIZYMPNGAI-UHFFFAOYSA-N,[Zn+2],300,12122-67-7,[Zn++].[S-]C(=S)NCCNC([S-])=S
2432,2737,Molecule_2737,[Zn+2].CN(C)C([S-])=S.CN(C)C([S-])=S,3,CN(C)C(S)=S,InChI=1S/C3H7NS2/c1-4(2)3(5)6/h1-2H3(H56),MZGNSEAPZQGJRB-UHFFFAOYSA-N,[Zn+2],300,137-30-4,[Zn++].CN(C)C([S-])=S.CN(C)C([S-])=S


#### Removing Duplicates 

In [13]:
filter = ['CASRN', 'Canonical_QSARr']

knimeDF = merge_knime[filter].drop_duplicates(subset= ['Canonical_QSARr']) 
knimeDF.head()

,CASRN,Canonical_QSARr
0,1655500-83-6,CC(CC=C(C)C)C1CC1(C)CO
1,109-88-6,CO
2,77-78-1,COS(=O)(=O)OC
3,2082-81-7,CC(=C)C(=O)OCCCCOC(=O)C(C)=C
4,75-33-2,CC(C)S


In [14]:
knimeDF.shape

(2286, 2)

In [17]:
combine_knime = pd.read_csv("../data/skin-sensitization/combined.csv")
combine_knime.head()


,CASRN,SMILES,Activity
0,1655500-83-6,CC(CC=C(C)C)C1CC1(C)CO,1
1,109-88-6,CO,1
2,77-78-1,COS(=O)(=O)OC,1
3,2082-81-7,CC(=C)C(=O)OCCCCOC(=O)C(C)=C,1
4,75-33-2,CC(C)S,1


In [18]:
combine_knime.shape

(4225, 3)

In [20]:
unique_knimeDF = combine_knime.drop_duplicates(subset= ['SMILES'], keep= False)
unique_knimeDF.shape

(3695, 3)

In [21]:
print('Positive class = ',unique_knimeDF[unique_knimeDF['Activity'] == 1].shape)
print('Negative class = ',unique_knimeDF[unique_knimeDF['Activity'] == 0].shape)

Positive class =  (2021, 3)
Negative class =  (1674, 3)


#### File saving 

In [25]:
file_name = f"QSAR_KNIME_Skin_Sensetization{datetime.datetime.now().isoformat()}"
file_path = "../models/qsar_knime" 
file = os.path.join(file_path, file_name)
pickle.dump(unique_knimeDF, open(file, 'wb'))